<a href="https://colab.research.google.com/github/lasmit17/SpotifyProject/blob/master/XGBoost_Collab_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import xgboost as xgb

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, auc, average_precision_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [7]:
luke_df_2016 = pd.read_csv(f'Luke_2016_Top_Songs.csv')
luke_df_2017 = pd.read_csv(f'Luke_2017_Top_Songs.csv')
luke_df_2018 = pd.read_csv(f'Luke_2018_Top_Songs.csv')
luke_df_2019 = pd.read_csv(f'Luke_2019_Top_Songs.csv')
luke_df_2020 = pd.read_csv(f'Luke_2020_Top_Songs.csv')

jp_df_2017 = pd.read_csv(f'jp_2017_Top_Songs.csv')
jp_df_2018 = pd.read_csv(f"jp_2018_Top_Songs.csv")
jp_df_2019 = pd.read_csv(f"jp_2019_Top_Songs.csv")
jp_df_2020 = pd.read_csv(f"jp_2020_Top_Songs.csv")

fabian_df_2018 = pd.read_csv(f"FP_2018_Top_Songs.csv")
fabian_df_2019 = pd.read_csv(f"fp_2019_Top_Songs.csv")
fabian_df_2020 = pd.read_csv(f"FP_2020_Top_Songs.csv")
fabian_df_2017 = pd.read_csv(f"FP_2017_Top_Songs.csv")

In [3]:
def get_years(df):
    years = []
    for date in df['release_date'].values:
        if '-' in date:
            years.append(date.split('-')[0])
        else:
            years.append(date)
    df['release_year'] = years
    return df

In [8]:
luke_df_2016 = get_years(luke_df_2016)
luke_df_2017 = get_years(luke_df_2017)
luke_df_2018 = get_years(luke_df_2018)
luke_df_2019 = get_years(luke_df_2019)
luke_df_2020 = get_years(luke_df_2020)

jp_df_2017 = get_years(jp_df_2017)
jp_df_2018 = get_years(jp_df_2018)
jp_df_2019 = get_years(jp_df_2019)
jp_df_2020 = get_years(jp_df_2020)

fabian_df_2018 = get_years(fabian_df_2018)
fabian_df_2019 = get_years(fabian_df_2019)
fabian_df_2020 = get_years(fabian_df_2020)
fabian_df_2017 = get_years(fabian_df_2017)

In [9]:
#combining each person's data
luke_df_concat = pd.concat([luke_df_2016, luke_df_2017, luke_df_2018, luke_df_2019, luke_df_2020], ignore_index=True, axis=0)
luke_df_concat['users_name'] = "Luke"

jp_df_concat = pd.concat([jp_df_2017, jp_df_2018, jp_df_2019, jp_df_2020], ignore_index=True, axis=0)
jp_df_concat['users_name'] = "JP"

fabian_df_concat = pd.concat([fabian_df_2017, fabian_df_2018, fabian_df_2019, fabian_df_2020], ignore_index=True, axis=0)
fabian_df_concat['users_name'] = "Fabian"

#Combining every data frame together into one
all_df = pd.concat([luke_df_concat, jp_df_concat, fabian_df_concat], ignore_index=True, axis=0)

all_df["release_year"] = pd.to_numeric(all_df["release_year"])

In [10]:
def convert_scale(df, col):
    df[col + '_old'] = df[col]
    new_max = 1
    new_min = 0
    new_range = new_max-new_min
    max_val = df[col].max()
    min_val=df[col].min()
    val_range = max_val - min_val
    df[col]=df[col].apply(lambda x: (((x-min_val)*new_range)/val_range)+new_min)
    return

#Setting the numerical spotify features
numeric_spotify_features = ['energy',
    'valence',
    'danceability',
    'liveness',
    'speechiness',
    'instrumentalness',
    'acousticness',
    'loudness',
    'length',
    'popularity',
    'tempo',
    'release_year']


for col in numeric_spotify_features:
    convert_scale(all_df, col)

#Now we need to address the categorical variables. First, we will utilize One Hot Encoder
ohe = OneHotEncoder()
ohe_results = ohe.fit_transform(all_df[['time_signature', 'mode', 'key']])
onehot_df = pd.DataFrame(ohe_results.toarray(), columns=['time_signature_1', 'time_signature_2',
                                                     'time_signature_3', 'time_signature_4',
                                                     'mode_1', 'mode_2',
                                                     'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6',
                                                     'key_7', 'key_8', 'key_9', 'key_10', 'key_11', 'key_12'])
all_df = pd.concat([all_df, onehot_df], axis=1)

In [11]:
all_spotify_features = ['energy',
    'valence',
    'danceability',
    'liveness',
    'speechiness',
    'instrumentalness',
    'acousticness',
    'loudness',
    'length',
    'popularity',
    'tempo',
    'release_year',
    'time_signature_1',
    'time_signature_2',
    'time_signature_3',
    'time_signature_4',
    'mode_1',
    'mode_2',
    'key_1',
    'key_2',
    'key_3',
    'key_4',
    'key_5',
    'key_6',
    'key_7',
    'key_8',
    'key_9',
    'key_10',
    'key_11',
    'key_12']

In [15]:
all_df["users_name"] = all_df['users_name'].replace({"Luke":2,"JP":1,"Fabian":0})

X = all_df[all_spotify_features]
y = all_df['users_name']
data_dmatrix = xgb.DMatrix(data=X,label=y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

xg_reg = xgb.XGBClassifier(objective ='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10, num_classes = 3)
xg_reg.fit(X_train,y_train)


pred_xgb = xg_reg.predict(X_test)

mse_xgb = np.sqrt(mean_squared_error(y_test, pred_xgb))
print("MSE: %f" % (mse_xgb))

xgb_initial_score = cross_val_score(xg_reg, X, y, cv=10)
xgb_initial_score

xgb_initial_accuracy = accuracy_score(y_test, pred_xgb)
xgb_initial_accuracy


MSE: 0.938083


0.6738461538461539